# Vector space model - Improved instantiation

## TF-IDF

- Improve quality of weight of term by downweighting frequently occurring (hence less informative) terms.
- $x_i = c(w_i, d) \cdot \text{IDF}(w_i) $ where $c(w_i, d)$ is count of word $w_i$ in document $d$.



## IDF weighting: Penalize popular terms

$\text{IDF}(w) = \log\left( \frac{M + 1}{k} \right)$

- $M$ is the total number of documents in the collection 
- $k$ is total number of documents containing $w$ (doc frequency).
- Max value at $\log\left(M + 1\right)$.
- Min value at $\log\left(\frac{M + 1}{M}\right)$.

## TF transformation

- Originally, $f(q, d) = \sum_{w \in q \cap d} c(w, q) \cdot c(w, d) \log \left(\frac{M + 1}{df(w)} \right)$.

- Each occurrence of $w$ has the same weight. 
  - Undesirable because if $w$ appears frequently in $d$, any additional times it occur does not provide much more information than before.
- Want to adjust the contribution based on the count.


### BM25

- $y = \frac{(k + 1)x}{x + k}$ where $x = c(w, d)$ and $y$ is the new weight.
- $k$ is a parameter to vary.
- Different values of $k$ corresponds to different model.
  - $k = 0$ is bit vector.
- Upperbounds/limits weight of term. 
  - Limits influence of individual terms. Prevents spammer from spamming individual terms and overrun the system.



# Document length



## Document length normalization

- Penalize long document
  - Longer document more likely to match any query.
  - But need to avoid over-penalization.
- Long document because
  - uses more words $\implies$ more penalization.
  - more content $\implies$ less penalization.
  
- Pivoted length normalizer: average doc length as "pivot".
  - Normalizer = 1 if $\vert d \vert = $ average doc length ($\text{avdl}$).
  - Normalizer = $1 - b + b\frac{\vert d \vert}{\text{avdl}} $ where $b \in [0, 1]$.
  - $b = 0 \implies $ normalizer = 1, so penalization.
  - Larger $b \implies $ more penalization.
  
# Well known models

## Pivoted length normalization VSM (Singhal et al 96)


\begin{equation}
f(q, d) = \underbrace{ \sum_{w \in q \cap d} }_{\text{Sum over common terms}}
\underbrace{
c(w, q)
}_{\text{Query word count} }
\underbrace{
\overbrace{
\frac{ \overbrace{ \ln \left(1 + \ln (1 + c(w, d)) \right) }^{ \text{Double log down-weighting of doc word count } }  }
{ 
\underbrace{1 - b + b \frac{\vert d \vert }{\text{avdl}}}_{\text{Document length normalization} } }
}^{\text{TF transformation}}
\underbrace{\log \frac{M  + 1}{df(w)}}_{IDF(w)}
}_{ \text{Weighted document word count} }
\end{equation}

This basically introduces a double logarithm transformation on the count and normalize it using document normalization.


## BM25 / Okapi (Robertson et al)


\begin{equation}
f(q, d) = \sum_{w \in q \cap d} c(w, q)
\underbrace{
\frac{(k + 1) c(w, d) }{c(w, d) + k \left(1 - b + b \frac{\vert d \vert }{\text{avdl}} \right)}
}_{\text{BM25 transformation } \leq k + 1}
\log \frac{M + 1}{df(w)}
\end{equation}

This form is similar to the previous pivoted length normalization VSM except that the document term weighting is performed differently.

# Further improvements to VSM

- Improve instantiation of dimension
  - Stem words, stop word removal, phrases, latent semantic analysis (LSI), character n-grams...
  - BOW with phrases is often sufficient.
  - Language-specific and domain-specific tokenization is important to ensure "normalization of terms".
  
- Improved instantiation of similarity function
  - Dot product with appropriate term weighting seems to still work the best.

# Typical IR architecture

- Tokenizer
  - Normalize lexical units: Ensure words with similar meaning gets mapped to same indexing term.
  - Stemming: Map all infectional forms to same root form. E.g., Computer, computation, computing -> compute.
  
- Indexer (offline)
  - Convert documents to data structures to enable fast search (precompute as much as we can).
  - Inverted index, document index etc.

- Scorer (online)
- Feedback (offline and online)


## Inverted index

   <img src="https://nlp.stanford.edu/IR-book/html/htmledition/img54.png"/>
  
- Score can be computed easily from the statistics stored in the indexed without having to scan the documents on the fly.
- Relevant documents can be retrieved quickly from the index.
- Works because of word distributions in text - Zipf law.
- A few words appear very frequently in text but most words are rare.
- Frequent words in one corpus may be rare in another.

### Zipf's law

- rank * frequency of word $\approx$ constant - $F(w) = \frac{C}{r(w)^\alpha}, \alpha \approx 1, C \approx 0.1 $.

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ac/Zipf_30wiki_en_labels.png" width=500 height=500 />

### Data structures

- Dictionary
  - Fast random access.
  - Preferred to be in memory.
  - Hash-table, B-Tree, trie, ...
- Postings
  - Sequential access is expected.
  - Large, so store on disk.
  - May contain docID, term freq, term pos, etc.
  - Compress is desirable.

# Index compression


### Compressing word count

A form of variable length encoding where we use few bits too encode small numbers and more bits to encode large ones. Due to phenomenon behind Zipf's law, most words tend to appear infrequently, so their word counts and the number of document they appear in will tend to be small integers. Encoding them this way allows us to save memory.

### Compressing document ID

Furthermore, for document ID (docID), in the inverted index, we don't store the docIDs of the document containing a word. Instead, we sort the docID, store the first one, and then store the adjacent differences (known as "d-gap"). This way, we keep the numbers small, and they can in turn also be compressed efficiently using the scheme above.

## Integer compression

We can reduce memory by compressing the integers stored in the index using variable length encoding.

There are a few common encoding schemes,

1. Binary
2. Unary
3. Gamma
4. Delta

### Binary

This is just the binary string representation of the integer without any 0s as prefix.

### Unary

$x \ge 1$ is coded as $x - 1$ one bits followed by a 0. 

This is aggressive in compressing small numbers but disastrous when we encounter a large number.

### Gamma ($\gamma$) code

This is a less aggressive form of unary coding.

To encode $x$, 

Unary code $\underbrace{ 1 + \lfloor \log x \rfloor }_{k = \text{ # of bits in binary representation of } x} $ then concatenate with
binary code of 
$ \underbrace{ x - 
\underbrace{2^{ \lfloor \log x \rfloor }}_{\text{Largest 2 power } \leq x}}_{
P }$ using $k$ bits (padding with 0s if required).

Note that $\lfloor \log x \rfloor$ is exactly the number of 1s in the unary portion because the unary portion is coded as $1 + \lfloor \log x \rfloor - 1 = \lfloor \log x \rfloor$ number of 1s followed by a 0.

To decode, compute $x = 2^{\lfloor \log x \rfloor } + P $.

**Note**: All the above operations can be computed very easily and efficiently using bit-level operations.

An alternative given in the textbook (and IMO better unless $P$ can be compressed efficiently) is to store $k - 1$ 0s followed by a 1 and then followed by the binary representation of $x$. The 1 act as a delimiter and the number of 0s $+ 1$ tells us how many bits to read after that to get the binary representation of $x$.


### Delta ($\delta$) code

Same as $\gamma$ code but replace unary coding part with $\gamma$ code.

This is a less aggressive version of $\gamma$ code.


<font color="red">Note:</font> that we cannot code for 0 under all these schemes.

# Fast search using inverted index

- We can use inverted index to accumulate scores for documents matching query terms.
- We need to first retrieve the docID their corresponding statistics of documents matching the terms in the query.
- Then, allocate one accumulator (variable) for each document to store its accumulated score.
- After all the scores have been accumulated, we can return the scores and docID.

## Why use inverted index
- Using inverted index to look up document actually allows us to exploit Zipf's law phenomenon to retrieve information about a relatively small set of documents.
- Storing the appropriate set of statistics about the terms and documents can allow us to quickly compute the scores for a wide range of ranking algorithms.
- Scaling up requires use of distributed file system and parallel processing and caching.